## 🧠 torchsummary

### ✅ 설치 방법
`pip install torchsummary`

### ✅ 기본 사용법
- `input_size`: (channels, height, width)

```python
from torchsummary import summary

# 예: 모델 정의 후
model = MLP().to(DEVICE)

# summary 호출 (예: CIFAR10 이미지 3x32x32)
summary(model, input_size=(3, 32, 32))
```


In [ ]:
# !pip install torchsummary

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
import os
import json
import platform
import pkg_resources
from sklearn.metrics import precision_score, recall_score, f1_score
from torchsummary import summary

# GPU 설정
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 하이퍼파라미터 설정
BATCH_SIZE = 128
EPOCHS = 10
LEARNING_RATE = 0.001
WEIGHT_DECAY = 1e-5
DROPOUT_RATE = 0.3
INPUT_SIZE = 3 * 32 * 32
NUM_CLASSES = 10
MODEL_PATH = './best_model.pth'
PATIENCE = 3
SPLIT_RATIO = 0.8

# CIFAR10 데이터셋의 평균 및 표준편차 (정규화 기준)
CIFAR10_MEAN = (0.4914, 0.4822, 0.4465)
CIFAR10_STD = (0.2023, 0.1994, 0.2010)

# 학습 데이터 증강 설정 (후반 학습용)
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),
    transforms.RandomRotation(5),
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD)
])

# 테스트 데이터 변환 (데이터 증강 없이 정규화만 적용)
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD)
])

# 데이터셋 로드
full_train_dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root="./data", train=False, transform=transform_test, download=True)

# 학습 및 검증 데이터셋 분리
train_size = int(SPLIT_RATIO * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])
val_dataset.dataset.transform = transform_test

# DataLoader 정의
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# 모델 정의 (MLP)
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(INPUT_SIZE, 512), nn.BatchNorm1d(512), nn.ReLU(), nn.Dropout(DROPOUT_RATE),
            nn.Linear(512, 256), nn.BatchNorm1d(256), nn.ReLU(), nn.Dropout(DROPOUT_RATE),
            nn.Linear(256, 256), nn.BatchNorm1d(256), nn.ReLU(), nn.Dropout(DROPOUT_RATE),
            nn.Linear(256, 128), nn.BatchNorm1d(128), nn.ReLU(),
            nn.Linear(128, NUM_CLASSES)
        )

    def forward(self, x):
        return self.model(x)

# 모델 초기화
model = MLP().to(DEVICE)

# summary 호출 (예: CIFAR10 이미지 3x32x32)
summary(model, input_size=(3, 32, 32))

# 모델 불러오기
def load_model():
    if os.path.exists(MODEL_PATH):
        model.load_state_dict(torch.load(MODEL_PATH))
        print('Model loaded from checkpoint.')
    else:
        print('No checkpoint found, training from scratch.')
    return model

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

best_val_acc = 0
patience_counter = 0

# 학습 함수 정의
def train():
    global best_val_acc, patience_counter
    model.train()
    for epoch in range(EPOCHS):

        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        scheduler.step()

        avg_loss = running_loss / len(train_loader)
        val_loss, val_acc = validate()

        print(f"Epoch [{epoch+1}/{EPOCHS}], Train Loss: {avg_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

        if val_acc > best_val_acc:  # Early Stopping 체크
            best_val_acc = val_acc
            patience_counter = 0
            torch.save(model.state_dict(), MODEL_PATH)
            print(f"New Best Model Saved! Validation Accuracy: {best_val_acc:.2f}%")
        else:
            patience_counter += 1
            if patience_counter >= PATIENCE:
                print("Early stopping triggered.")
                break

def validate():
    model.eval()
    correct = 0
    total = 0
    val_loss_total = 0.0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss_total += loss.item()

            _, predicted = torch.max(outputs, 1) # 반환:(최댓값, 인덱스)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = val_loss_total / len(val_loader)
    val_acc = 100 * correct / total

    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')

    print(f"Validation - Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

    model.train()
    return val_loss, val_acc


def test():
    model.load_state_dict(torch.load(MODEL_PATH))
    model.eval()
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')

    print(f'Test Accuracy: {accuracy:.2f}%')
    print(f'Test Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')


# 최적의 하이퍼파라미터 정보와 환경 정보 저장 함수
def save_study_info(filename="best_study_info.json"):

    # Python 및 라이브러리 정보 가져오기
    python_version = platform.python_version()
    platform_info = platform.platform()
    installed_packages = {pkg.key: pkg.version for pkg in pkg_resources.working_set}

    # 데이터셋 정보 수집
    dataset_name = 'CIFAR10'  # 현재 사용 중인 데이터셋 이름
    dataset_info = {
        "dataset_name": dataset_name,
        "train_size": len(train_dataset),
        "validation_size": len(val_dataset),
        "test_size": len(test_dataset),
        "image_size": (32, 32),  # CIFAR-10 이미지 크기
        "num_classes": NUM_CLASSES
    }

    # 모델 정보 수집
    model_info = {
        "model_name": model.__class__.__name__,
        "model_structure": str(model),
        "input_size": INPUT_SIZE,
        "num_classes": NUM_CLASSES
    }

    # 정보 저장
    data = {
        "python_version": python_version,
        "platform_info": platform_info,
        "installed_packages": installed_packages,
        "dataset_info": dataset_info,
        "model_info": model_info
    }

    with open(filename, "w") as f:
        json.dump(data, f, indent=4)

    print(f"Best study info saved to {filename}")


if __name__ == "__main__":
    # load_model()
    train()
    test()

    # 최적의 정보 저장
    save_study_info()

<ipython-input-2-0d44eb106e74>:12: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
100%|██████████| 170M/170M [00:13<00:00, 12.8MB/s]


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                 [-1, 3072]               0
            Linear-2                  [-1, 512]       1,573,376
       BatchNorm1d-3                  [-1, 512]           1,024
              ReLU-4                  [-1, 512]               0
           Dropout-5                  [-1, 512]               0
            Linear-6                  [-1, 256]         131,328
       BatchNorm1d-7                  [-1, 256]             512
              ReLU-8                  [-1, 256]               0
           Dropout-9                  [-1, 256]               0
           Linear-10                  [-1, 256]          65,792
      BatchNorm1d-11                  [-1, 256]             512
             ReLU-12                  [-1, 256]               0
          Dropout-13                  [-1, 256]               0
           Linear-14                  [

## 🧠 torchinfo

### ✅ 설치 방법
`pip install torchinfo`

### ✅ 기본 사용법
- `input_size`: 배치 차원 포함 (batch_size, channels, height, width)

```python
from torchinfo import summary

# 모델 생성
model = MLP().to(DEVICE)

# summary 호출
summary(model, input_size=(BATCH_SIZE, 3, 32, 32))
```


In [ ]:
# !pip install torchinfo

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
import os
import json
import platform
import pkg_resources
from sklearn.metrics import precision_score, recall_score, f1_score
from torchinfo import summary

# GPU 설정
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 하이퍼파라미터 설정
BATCH_SIZE = 128
EPOCHS = 10
LEARNING_RATE = 0.001
WEIGHT_DECAY = 1e-5
DROPOUT_RATE = 0.3
INPUT_SIZE = 3 * 32 * 32
NUM_CLASSES = 10
MODEL_PATH = './best_model.pth'
PATIENCE = 3
SPLIT_RATIO = 0.8

# CIFAR10 데이터셋의 평균 및 표준편차 (정규화 기준)
CIFAR10_MEAN = (0.4914, 0.4822, 0.4465)
CIFAR10_STD = (0.2023, 0.1994, 0.2010)

# 학습 데이터 증강 설정 (후반 학습용)
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),
    transforms.RandomRotation(5),
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD)
])

# 테스트 데이터 변환 (데이터 증강 없이 정규화만 적용)
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD)
])

# 데이터셋 로드
full_train_dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root="./data", train=False, transform=transform_test, download=True)

# 학습 및 검증 데이터셋 분리
train_size = int(SPLIT_RATIO * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])
val_dataset.dataset.transform = transform_test

# DataLoader 정의
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# 모델 정의 (MLP)
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(INPUT_SIZE, 512), nn.BatchNorm1d(512), nn.ReLU(), nn.Dropout(DROPOUT_RATE),
            nn.Linear(512, 256), nn.BatchNorm1d(256), nn.ReLU(), nn.Dropout(DROPOUT_RATE),
            nn.Linear(256, 256), nn.BatchNorm1d(256), nn.ReLU(), nn.Dropout(DROPOUT_RATE),
            nn.Linear(256, 128), nn.BatchNorm1d(128), nn.ReLU(),
            nn.Linear(128, NUM_CLASSES)
        )

    def forward(self, x):
        return self.model(x)

# 모델 초기화
model = MLP().to(DEVICE)

# summary 호출
print(summary(model, input_size=(BATCH_SIZE, 3, 32, 32)))

# 모델 불러오기
def load_model():
    if os.path.exists(MODEL_PATH):
        model.load_state_dict(torch.load(MODEL_PATH))
        print('Model loaded from checkpoint.')
    else:
        print('No checkpoint found, training from scratch.')
    return model

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

best_val_acc = 0
patience_counter = 0

# 학습 함수 정의
def train():
    global best_val_acc, patience_counter
    model.train()
    for epoch in range(EPOCHS):

        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        scheduler.step()

        avg_loss = running_loss / len(train_loader)
        val_loss, val_acc = validate()

        print(f"Epoch [{epoch+1}/{EPOCHS}], Train Loss: {avg_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

        if val_acc > best_val_acc:  # Early Stopping 체크
            best_val_acc = val_acc
            patience_counter = 0
            torch.save(model.state_dict(), MODEL_PATH)
            print(f"New Best Model Saved! Validation Accuracy: {best_val_acc:.2f}%")
        else:
            patience_counter += 1
            if patience_counter >= PATIENCE:
                print("Early stopping triggered.")
                break

def validate():
    model.eval()
    correct = 0
    total = 0
    val_loss_total = 0.0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss_total += loss.item()

            _, predicted = torch.max(outputs, 1) # 반환:(최댓값, 인덱스)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = val_loss_total / len(val_loader)
    val_acc = 100 * correct / total

    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')

    print(f"Validation - Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

    model.train()
    return val_loss, val_acc


def test():
    model.load_state_dict(torch.load(MODEL_PATH))
    model.eval()
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')

    print(f'Test Accuracy: {accuracy:.2f}%')
    print(f'Test Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')


# 최적의 하이퍼파라미터 정보와 환경 정보 저장 함수
def save_study_info(filename="best_study_info.json"):

    # Python 및 라이브러리 정보 가져오기
    python_version = platform.python_version()
    platform_info = platform.platform()
    installed_packages = {pkg.key: pkg.version for pkg in pkg_resources.working_set}

    # 데이터셋 정보 수집
    dataset_name = 'CIFAR10'  # 현재 사용 중인 데이터셋 이름
    dataset_info = {
        "dataset_name": dataset_name,
        "train_size": len(train_dataset),
        "validation_size": len(val_dataset),
        "test_size": len(test_dataset),
        "image_size": (32, 32),  # CIFAR-10 이미지 크기
        "num_classes": NUM_CLASSES
    }

    # 모델 정보 수집
    model_info = {
        "model_name": model.__class__.__name__,
        "model_structure": str(model),
        "input_size": INPUT_SIZE,
        "num_classes": NUM_CLASSES
    }

    # 정보 저장
    data = {
        "python_version": python_version,
        "platform_info": platform_info,
        "installed_packages": installed_packages,
        "dataset_info": dataset_info,
        "model_info": model_info
    }

    with open(filename, "w") as f:
        json.dump(data, f, indent=4)

    print(f"Best study info saved to {filename}")


if __name__ == "__main__":
    # load_model()
    train()
    test()

    # 최적의 정보 저장
    save_study_info()

Layer (type:depth-idx)                   Output Shape              Param #
MLP                                      [128, 10]                 --
├─Sequential: 1-1                        [128, 10]                 --
│    └─Flatten: 2-1                      [128, 3072]               --
│    └─Linear: 2-2                       [128, 512]                1,573,376
│    └─BatchNorm1d: 2-3                  [128, 512]                1,024
│    └─ReLU: 2-4                         [128, 512]                --
│    └─Dropout: 2-5                      [128, 512]                --
│    └─Linear: 2-6                       [128, 256]                131,328
│    └─BatchNorm1d: 2-7                  [128, 256]                512
│    └─ReLU: 2-8                         [128, 256]                --
│    └─Dropout: 2-9                      [128, 256]                --
│    └─Linear: 2-10                      [128, 256]                65,792
│    └─BatchNorm1d: 2-11                 [128, 256]              

## 🧠 torchviz

### ✅ 설치 방법
`pip install torchviz`

### ✅ 사용법 요약
- `make_dot()`을 사용해서 모델의 그래프를 생성

```python
from torchviz import make_dot

# 모델 및 더미 입력
model = MLP().to(DEVICE)
dummy_input = torch.randn(BATCH_SIZE, 3, 32, 32).to(DEVICE)  # 배치 크기 모델과 동일

# forward pass (output)
output = model(dummy_input)

# 그래프 생성
dot = make_dot(output, params=dict(model.named_parameters()))

# 저장 (PDF or PNG 등으로)
dot.render("model_graph", format="png")  # model_graph.png 생성됨
```


In [ ]:
# !pip install torchviz

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
import os
import json
import platform
import pkg_resources
from sklearn.metrics import precision_score, recall_score, f1_score
from torchviz import make_dot

# GPU 설정
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 하이퍼파라미터 설정
BATCH_SIZE = 128
EPOCHS = 10
LEARNING_RATE = 0.001
WEIGHT_DECAY = 1e-5
DROPOUT_RATE = 0.3
INPUT_SIZE = 3 * 32 * 32
NUM_CLASSES = 10
MODEL_PATH = './best_model.pth'
PATIENCE = 3
SPLIT_RATIO = 0.8

# CIFAR10 데이터셋의 평균 및 표준편차 (정규화 기준)
CIFAR10_MEAN = (0.4914, 0.4822, 0.4465)
CIFAR10_STD = (0.2023, 0.1994, 0.2010)

# 학습 데이터 증강 설정 (후반 학습용)
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),
    transforms.RandomRotation(5),
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD)
])

# 테스트 데이터 변환 (데이터 증강 없이 정규화만 적용)
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD)
])

# 데이터셋 로드
full_train_dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root="./data", train=False, transform=transform_test, download=True)

# 학습 및 검증 데이터셋 분리
train_size = int(SPLIT_RATIO * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])
val_dataset.dataset.transform = transform_test

# DataLoader 정의
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# 모델 정의 (MLP)
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(INPUT_SIZE, 512), nn.BatchNorm1d(512), nn.ReLU(), nn.Dropout(DROPOUT_RATE),
            nn.Linear(512, 256), nn.BatchNorm1d(256), nn.ReLU(), nn.Dropout(DROPOUT_RATE),
            nn.Linear(256, 256), nn.BatchNorm1d(256), nn.ReLU(), nn.Dropout(DROPOUT_RATE),
            nn.Linear(256, 128), nn.BatchNorm1d(128), nn.ReLU(),
            nn.Linear(128, NUM_CLASSES)
        )

    def forward(self, x):
        return self.model(x)

# 모델 초기화
model = MLP().to(DEVICE)

# torchviz로 그래프 그리기

dummy_input = torch.randn(BATCH_SIZE, 3, 32, 32).to(DEVICE)  # 배치 크기 1

output = model(dummy_input) # forward pass (output)

dot = make_dot(output, params=dict(model.named_parameters())) # 그래프 생성

dot.render("model_graph", format="png")  # model_graph.png 생성됨



# 모델 불러오기
def load_model():
    if os.path.exists(MODEL_PATH):
        model.load_state_dict(torch.load(MODEL_PATH))
        print('Model loaded from checkpoint.')
    else:
        print('No checkpoint found, training from scratch.')
    return model

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

best_val_acc = 0
patience_counter = 0

# 학습 함수 정의
def train():
    global best_val_acc, patience_counter
    model.train()
    for epoch in range(EPOCHS):

        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        scheduler.step()

        avg_loss = running_loss / len(train_loader)
        val_loss, val_acc = validate()

        print(f"Epoch [{epoch+1}/{EPOCHS}], Train Loss: {avg_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

        if val_acc > best_val_acc:  # Early Stopping 체크
            best_val_acc = val_acc
            patience_counter = 0
            torch.save(model.state_dict(), MODEL_PATH)
            print(f"New Best Model Saved! Validation Accuracy: {best_val_acc:.2f}%")
        else:
            patience_counter += 1
            if patience_counter >= PATIENCE:
                print("Early stopping triggered.")
                break

def validate():
    model.eval()
    correct = 0
    total = 0
    val_loss_total = 0.0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss_total += loss.item()

            _, predicted = torch.max(outputs, 1) # 반환:(최댓값, 인덱스)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = val_loss_total / len(val_loader)
    val_acc = 100 * correct / total

    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')

    print(f"Validation - Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

    model.train()
    return val_loss, val_acc


def test():
    model.load_state_dict(torch.load(MODEL_PATH))
    model.eval()
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')

    print(f'Test Accuracy: {accuracy:.2f}%')
    print(f'Test Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')


# 최적의 하이퍼파라미터 정보와 환경 정보 저장 함수
def save_study_info(filename="best_study_info.json"):

    # Python 및 라이브러리 정보 가져오기
    python_version = platform.python_version()
    platform_info = platform.platform()
    installed_packages = {pkg.key: pkg.version for pkg in pkg_resources.working_set}

    # 데이터셋 정보 수집
    dataset_name = 'CIFAR10'  # 현재 사용 중인 데이터셋 이름
    dataset_info = {
        "dataset_name": dataset_name,
        "train_size": len(train_dataset),
        "validation_size": len(val_dataset),
        "test_size": len(test_dataset),
        "image_size": (32, 32),  # CIFAR-10 이미지 크기
        "num_classes": NUM_CLASSES
    }

    # 모델 정보 수집
    model_info = {
        "model_name": model.__class__.__name__,
        "model_structure": str(model),
        "input_size": INPUT_SIZE,
        "num_classes": NUM_CLASSES
    }

    # 정보 저장
    data = {
        "python_version": python_version,
        "platform_info": platform_info,
        "installed_packages": installed_packages,
        "dataset_info": dataset_info,
        "model_info": model_info
    }

    with open(filename, "w") as f:
        json.dump(data, f, indent=4)

    print(f"Best study info saved to {filename}")


if __name__ == "__main__":
    # load_model()
    train()
    test()

    # 최적의 정보 저장
    save_study_info()

Validation - Precision: 0.4407, Recall: 0.4458, F1 Score: 0.4352
Epoch [1/10], Train Loss: 1.7784, Val Loss: 1.5436, Val Acc: 44.55%
New Best Model Saved! Validation Accuracy: 44.55%
Validation - Precision: 0.4683, Recall: 0.4678, F1 Score: 0.4579
Epoch [2/10], Train Loss: 1.5808, Val Loss: 1.4770, Val Acc: 46.83%
New Best Model Saved! Validation Accuracy: 46.83%
Validation - Precision: 0.4917, Recall: 0.4952, F1 Score: 0.4902
Epoch [3/10], Train Loss: 1.4953, Val Loss: 1.4091, Val Acc: 49.52%
New Best Model Saved! Validation Accuracy: 49.52%
Validation - Precision: 0.5044, Recall: 0.5034, F1 Score: 0.5010
Epoch [4/10], Train Loss: 1.4380, Val Loss: 1.3806, Val Acc: 50.31%
New Best Model Saved! Validation Accuracy: 50.31%
Validation - Precision: 0.5165, Recall: 0.5162, F1 Score: 0.5128
Epoch [5/10], Train Loss: 1.3930, Val Loss: 1.3466, Val Acc: 51.66%
New Best Model Saved! Validation Accuracy: 51.66%
Validation - Precision: 0.5325, Recall: 0.5324, F1 Score: 0.5308
Epoch [6/10], Train 